### Libs

In [15]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from copy import copy
import keras

### Load data

In [16]:
data = pd.read_csv('sudoku.csv')
quizzes, solutions = data["quizzes"], data["solutions"]

### Data preprocessing

In [77]:
sudoku_unsolved, sudoku_solved = [], []
for i in tqdm(range(len(quizzes)), desc="Formatting data"):
    sudoku_unsolved.append(np.array([int(x)/9.0-.5 for x in quizzes[i]]).reshape(9,9,1))
    sudoku_solved.append(np.array([int(x)-1 for x in solutions[i]]).reshape(81,1))
    
sudoku_unsolved, sudoku_solved = np.array(sudoku_unsolved), np.array(sudoku_solved)
print("Sudokus shape:", sudoku_unsolved.shape)
print("Solutions shape:", sudoku_solved.shape)

Formatting data: 100%|██████████| 1000000/1000000 [00:33<00:00, 29469.31it/s]


Sudokus shape: (1000000, 9, 9, 1)
Solutions shape: (1000000, 81, 1)


In [18]:
x_train, x_test, y_train, y_test = train_test_split(sudoku_unsolved, sudoku_solved, test_size=0.0001)

### Model

In [19]:
model = keras.models.Sequential()

model.add(keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu', padding='same', input_shape=(9,9,1)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(128, kernel_size=(1,1), activation='relu', padding='same'))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(81*9))
model.add(keras.layers.Reshape((-1, 9)))
model.add(keras.layers.Activation('softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 9, 9, 64)          640       
                                                                 
 batch_normalization (BatchN  (None, 9, 9, 64)         256       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 9, 9, 64)          36928     
                                                                 
 batch_normalization_1 (Batc  (None, 9, 9, 64)         256       
 hNormalization)                                                 
                                                                 
 conv2d_2 (Conv2D)           (None, 9, 9, 128)         8320      
                                                                 
 flatten (Flatten)           (None, 10368)             0

### Train model

In [20]:
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer)

model.fit(x_train, y_train, batch_size=32, epochs=2)

# Saving model to avoid retraining it each time
model.save('saved_models/model.h5')
print("Model saved")

Epoch 1/2
31247/31247 [==============================] - 1235s 40ms/step - loss: 0.4350
Epoch 2/2
31247/31247 [==============================] - 1137s 36ms/step - loss: 0.3580
Model saved


### Or load pretrained model

In [21]:
model = keras.models.load_model('./saved_models/model.h5')

### Test accuracy

In [32]:
print("Testing model...")
predictions = model.predict(x_test)

total_correct_box = 0
total_correct_emptyBox = 0
total_zero_boxes = 0
total_correct_puzzles = 0

for i in tqdm(range(len(predictions)), desc="Calculating accuracy"):
    formatted_pred = (np.argmax(predictions[i], axis=1)+1).reshape(81)
    formatted_solution = (y_test[i]+1).reshape(81)
    correct_boxes = sum(formatted_pred == formatted_solution)
    puzzle_zero_count = np.count_nonzero((x_test[0].reshape(81)+.5)*9 == 0)
    
    total_correct_box += correct_boxes
    total_correct_emptyBox += correct_boxes-(81-puzzle_zero_count)
    total_zero_boxes += puzzle_zero_count
    total_correct_puzzles += correct_boxes == 81

print(f"Total puzzle accuracy: {100*total_correct_puzzles/len(predictions):.2f}%")
print(f"Empty box accuracy: {100*total_correct_emptyBox/total_zero_boxes:.2f}%")
print(f"Global box accuracy: {100*total_correct_box/(len(predictions)*81):.2f}%")

Testing model...
4/4 [==============================] - 0s 28ms/step


Calculating accuracy: 100%|██████████| 100/100 [00:00<00:00, 28393.61it/s]

Total puzzle accuracy: 0.00%
Empty box accuracy: 69.19%
Global box accuracy: 82.12%


### Solve sudokus box by box

In [23]:
norm = lambda x: x/9.0-.5
denorm = lambda x: (x+.5)*9.0

def get_next_step(sudoku, confidence):
    sudoku = sudoku.reshape(9,9,1)
    preds = model.predict(np.array([sudoku,]), verbose=0).reshape(81,9)
    max_index, max_prob = -1, -1
    for i, val in enumerate(denorm(sudoku).reshape(81)):
        if val != 0: continue
        max_prob_index = np.argmax(preds[i])
        max_prob_value = preds[i][max_prob_index]
        if (max_prob_value>max_prob):
            max_index, max_prob = max_prob_index+i*9, max_prob_value
            
    if max_prob == -1: return sudoku.reshape(9,9), -1, confidence
    pos, val = int(max_index//9), max_index%9+1
    sudoku.reshape(81)[pos] = norm(val)
    return sudoku.reshape(9,9), max_prob, confidence*max_prob
    
    
def solve_sudoku(sudoku):    
    sudoku, prob, confidence = copy(sudoku).reshape(9,9), 0, 1
    while prob != -1:
        sudoku, prob, confidence = get_next_step(sudoku, confidence)
    return denorm(sudoku), confidence

### Check sudoku validity

In [71]:
def sudoku_to_int(sudoku):
    return [[int(x) for x in row] for row in sudoku]

def is_valid_sudoku(sudoku):
    
    sudoku = sudoku_to_int(sudoku)
    
    # Row check
    if sum([len(set(row)) == 9 for row in sudoku]) != 9:
        return False
    
    # Column check
    if sum([len(set([row[i] for row in sudoku])) == 9 for i in range(9)]) != 9:
        return False
        
    # Cell check
    for i in range(3):
        for j in range(0,9,3):
            cell = []
            for k in range(i*3,(i+1)*3):
                cell += sudoku[k][j:j+3]
            if len(set(cell)) != 9:
                return False
            
    return True

### Testing new accuraty with this method

In [37]:
print("Calculating accuracy...")
n_correct, total_confidence = 0, 0
for i in range(len(x_test)):
    solved_sudoku, confidence = solve_sudoku(x_test[0])
    total_confidence += confidence
    if np.sum(solved_sudoku == y_test[0].reshape(9,9)+1) == 81:
        n_correct+=1
        
print(f"Model accuracy: {100*n_correct/len(x_test)}%\nAverage confidence: {100*total_confidence/len(x_test):.2f}%")

Calculating accuracy...
Model accuracy: 100.0%
Average confidence: 97.80%


### Sudoku example (medium difficulty)

In [78]:
sudoku = norm(np.array([
    [3,0,0,4,0,1,6,2,0],
    [1,0,0,0,8,0,4,0,0],
    [0,0,5,0,2,0,8,3,0],
    [0,5,7,8,0,0,0,0,0],
    [0,0,0,7,0,0,5,0,3],
    [0,0,2,9,0,4,0,0,7],
    [4,8,0,5,3,0,0,1,0],
    [2,0,3,0,9,0,0,0,0],
    [0,7,0,0,0,6,0,9,0]
])).reshape(9,9,1)

solved, confidence = solve_sudoku(sudoku)
print(solved, f"\nModel confidence: {confidence*100:.2f}%")

[[3. 9. 8. 4. 7. 1. 6. 2. 5.]
 [1. 2. 6. 3. 8. 5. 4. 7. 9.]
 [7. 4. 5. 6. 2. 9. 8. 3. 1.]
 [6. 5. 7. 8. 1. 3. 9. 4. 2.]
 [9. 1. 4. 7. 6. 2. 5. 8. 3.]
 [8. 3. 2. 9. 5. 4. 1. 6. 7.]
 [4. 8. 9. 5. 3. 7. 2. 1. 6.]
 [2. 6. 3. 1. 9. 8. 7. 5. 4.]
 [5. 7. 1. 2. 4. 6. 3. 9. 8.]] 
Model confidence: 97.46%


### Sudoku example (Classic puzzle from the 2017 World Sudoku Grand Prix final recommended by Richard Stolk)
For more details click [here](https://gp.worldpuzzle.org/content/final-results-5), it is the sudoku #4 of the Sudoku Playoffs.

In [75]:
sudoku = norm(np.array([
    [0,0,0,0,0,6,0,4,0],
    [8,0,0,0,5,0,0,0,2],
    [0,7,0,4,0,0,9,0,0],
    [0,0,6,0,0,3,0,1,0],
    [5,0,0,0,0,0,0,0,3],
    [0,4,0,1,0,0,7,0,0],
    [0,0,3,0,0,9,0,6,0],
    [2,0,0,0,8,0,0,0,5],
    [0,1,0,7,0,0,0,0,0]
])).reshape(9,9,1)

solved, confidence = solve_sudoku(sudoku)
print(solved, f"\n\nModel confidence: {confidence*100:.2f}%")
print("Valid:", is_valid_sudoku(solved))

[[1. 5. 9. 2. 7. 6. 3. 4. 8.]
 [8. 3. 4. 9. 5. 1. 6. 7. 2.]
 [6. 7. 2. 4. 3. 8. 9. 5. 1.]
 [7. 2. 6. 8. 4. 3. 5. 1. 9.]
 [5. 9. 1. 6. 2. 7. 4. 8. 3.]
 [3. 4. 8. 1. 9. 5. 7. 2. 6.]
 [4. 8. 3. 5. 1. 9. 2. 6. 7.]
 [2. 6. 7. 3. 8. 4. 1. 9. 5.]
 [9. 1. 5. 7. 6. 2. 8. 3. 4.]] 

Model confidence: 14.88%
Valid: True
